In [1]:
import imapclient

imapclientOBJ = imapclient.IMAPClient('imap.gmail.com', ssl=True)


In [2]:
import getpass
def login():
    email = getpass.getpass('Enter Your Email: ')
    password = getpass.getpass('Enter Your secret Password: ')
    imapclientOBJ.login(email, password)
    print('You are sucessfully logged in')

In [3]:
login()

Enter Your Email:  ········
Enter Your secret Password:  ········


You are sucessfully logged in


In [4]:
imapclientOBJ.select_folder('INBOX', readonly=True)
UIDs = imapclientOBJ.search(['SINCE', '03-Jul-2024'])
UIDs[0]

3757

In [5]:
raw_messages = imapclientOBJ.fetch(UIDs[0], ['BODY[]', 'FLAGS'] )

In [6]:
raw_messages

defaultdict(dict,
            {3757: {b'SEQ': 3732,
              b'FLAGS': (),
              b'BODY[]': b'Delivered-To: jmldpp@gmail.com\r\nReceived: by 2002:a0c:fe87:0:b0:6b5:9e9:d891 with SMTP id d7csp2538786qvs;\r\n        Tue, 2 Jul 2024 20:14:50 -0700 (PDT)\r\nX-Google-Smtp-Source: AGHT+IH1a4zYZbfdd9fVRpwtLjm1J153bPjKqwXSdLlbDWiDsQNHwSjKJsOHv4oAGq0EIwyjf3dJ\r\nX-Received: by 2002:a25:2e4d:0:b0:dff:2e48:7aea with SMTP id 3f1490d57ef6-e036ec4b382mr12323766276.43.1719976489767;\r\n        Tue, 02 Jul 2024 20:14:49 -0700 (PDT)\r\nARC-Seal: i=1; a=rsa-sha256; t=1719976489; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=p70S4ALtxSafGntYAPvPvTGRp/AKK6IRGrHXXKE1NS0i28xG3lgvJC3G3JyCE7uiIm\r\n         EmCis6YTYBbs9Kp+4TE+obve0MRCwafSmTC+Na+jCtZW6qCjotfQZS99wV/oBFZBqSCA\r\n         d/H/qdTAh28GZ3ewEAD8E+n/GxsapvE+VEOEVJBWyhBjjnwUOaeUC1Uo9DbFYDxhs5W1\r\n         ztLDqEgS9WlrszDOvXS+Dsunkoc7xrPPJ0cR3VMOlFMx2LlRMEiA0AJcZqW0Navrytir\r\n         woxi4iaxMMdluSMLrRDEsHmwbGAXYz0F3e

In [15]:
import email
from email import policy
from email.parser import BytesParser

# This takes the raw_message as an arguement
def parse_email(raw_email):
    msg = BytesParser(policy=policy.default).parsebytes(raw_email)
    # Extract headers and body
    email_info = {
        "Subject": msg.get("Subject"),
        "From": msg.get("From"), 
        "To": msg.get("To"), 
        "Date": msg.get("Date")
    }
    # Extracting the email body
    if msg.is_multipart():
        # To get the plain text part 
        for part in msg.iter_parts():
            if part.get_content_type() == "text/plain":
                email_info["Body"] = part.get_content().strip()
                break
    else:
        # if not multipart get content directly
        email_info["Body"] = msg.get_content().strip()
    
    return email_info


In [17]:
type(raw_messages)

collections.defaultdict

In [18]:
raw_email = raw_messages.get('raw')

# Proceed only if raw_email is in bytes format
if isinstance(raw_email, bytes):
    parsed_email = parse_email(raw_email)
    for key, value in parsed_email.items():
        print(f'{key}: {value}')
else:
    print("Error: The raw email content is not in bytes format.")

Error: The raw email content is not in bytes format.


In [19]:
# Assuming the raw email content is in string format, we first convert it to bytes
raw_email = raw_messages.get('raw')

# Check if raw_email is a string and convert it to bytes if necessary
if isinstance(raw_email, str):
    raw_email = raw_email.encode('utf-8')

# Proceed to parse the email if it's now in bytes format
if isinstance(raw_email, bytes):
    parsed_email = parse_email(raw_email)
    for key, value in parsed_email.items():
        print(f'{key}: {value}')
else:
    print("Error: The raw email content is neither in bytes nor convertible to bytes.")


Error: The raw email content is neither in bytes nor convertible to bytes.


In [20]:
# Attempting to access the email content
raw_email_content = raw_messages.get('raw') or raw_messages.get('payload')  # Adjust based on the keys in raw_messages

# Check if the raw_email_content is a string, and convert to bytes if needed
if isinstance(raw_email_content, str):
    raw_email = raw_email_content.encode('utf-8')
elif isinstance(raw_email_content, bytes):
    raw_email = raw_email_content
else:
    raise TypeError("Unable to retrieve raw email content as string or bytes.")

# Parse the email if in bytes format
if isinstance(raw_email, bytes):
    parsed_email = parse_email(raw_email)
    for key, value in parsed_email.items():
        print(f'{key}: {value}')
else:
    print("Error: The raw email content is not accessible in bytes or convertible format.")


TypeError: Unable to retrieve raw email content as string or bytes.

In [21]:
# View the type of each item and try to locate the email content
for key, value in raw_messages.items():
    print(f"{key}: {type(value)}")


3757: <class 'dict'>


In [22]:
first_email = raw_messages[3757]
for inner_key, inner_value in first_email.items():
    print(f"{inner_key}: {type(inner_value)}")

b'SEQ': <class 'int'>
b'FLAGS': <class 'tuple'>
b'BODY[]': <class 'bytes'>


In [23]:
# Extract the raw email content from the bytes in BODY[]
raw_email_content = first_email[b'BODY[]']


In [24]:
raw_email_content

b'Delivered-To: jmldpp@gmail.com\r\nReceived: by 2002:a0c:fe87:0:b0:6b5:9e9:d891 with SMTP id d7csp2538786qvs;\r\n        Tue, 2 Jul 2024 20:14:50 -0700 (PDT)\r\nX-Google-Smtp-Source: AGHT+IH1a4zYZbfdd9fVRpwtLjm1J153bPjKqwXSdLlbDWiDsQNHwSjKJsOHv4oAGq0EIwyjf3dJ\r\nX-Received: by 2002:a25:2e4d:0:b0:dff:2e48:7aea with SMTP id 3f1490d57ef6-e036ec4b382mr12323766276.43.1719976489767;\r\n        Tue, 02 Jul 2024 20:14:49 -0700 (PDT)\r\nARC-Seal: i=1; a=rsa-sha256; t=1719976489; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=p70S4ALtxSafGntYAPvPvTGRp/AKK6IRGrHXXKE1NS0i28xG3lgvJC3G3JyCE7uiIm\r\n         EmCis6YTYBbs9Kp+4TE+obve0MRCwafSmTC+Na+jCtZW6qCjotfQZS99wV/oBFZBqSCA\r\n         d/H/qdTAh28GZ3ewEAD8E+n/GxsapvE+VEOEVJBWyhBjjnwUOaeUC1Uo9DbFYDxhs5W1\r\n         ztLDqEgS9WlrszDOvXS+Dsunkoc7xrPPJ0cR3VMOlFMx2LlRMEiA0AJcZqW0Navrytir\r\n         woxi4iaxMMdluSMLrRDEsHmwbGAXYz0F3eGYiInQRCfax4qZIzLiV/Zm29L0Aq7BzjHC\r\n         wA3g==\r\nARC-Message-Signature: i=1; a=rsa-sha256; c=rela

In [25]:
# Call your parsing function
parsed_email = parse_email(raw_email_content)

# Print out the structured email information
for key, value in parsed_email.items():
    print(f"{key}: {value}")

Subject: ISRO posted: Aditya-L1: Celebration of First Orbit Completion 🌞🛰️ ...
From: "X (formerly Twitter)" <info@twitter.com>
To: Jamiu Oladipupo <jmldpp@gmail.com>
Date: Wed, 03 Jul 2024 03:14:48 +0000
Body: Your Highlights
ISRO - @isro
Aditya-L1: Celebration of First Orbit Completion 🌞🛰️
Today, Aditya-L1 completed its first halo orbit around the Sun-Earth L1 point. Inserted on January 6, 2024, it took 178 days, to complete a revolution.

Today's station-keeping manoeuvre ensured its seamless transition into… https://t.co/yB6vZQpIvE
Read more at Twitter <https://twitter.com/isro/status/1808122572263117306>
  ----------
Jared Isaacman - @rookisaacman
Appreciate the coverage @PopSci 

https://t.co/RRlrAILKUn
Read more at Twitter <https://twitter.com/rookisaacman/status/1808180384867537011>
  ----------
Solace - @Solace_PR
Send ur Acct details now⭐

N3m Giveaway, Crediting first 5000 likes, Rt and follow Me. No wizkid Fc should skip ❤️🦅 Turkey x Sszn ⭐ https://t.co/LRtWectEsI
Read more 

In [26]:
import email
from email import policy
from email.parser import BytesParser

# Your email parsing function
def parse_email(raw_email):
    msg = BytesParser(policy=policy.default).parsebytes(raw_email)
    # Extract headers and body
    email_info = {
        "Subject": msg.get("Subject"),
        "From": msg.get("From"), 
        "To": msg.get("To"), 
        "Date": msg.get("Date")
    }
    # Extracting the email body
    if msg.is_multipart():
        # To get the plain text part 
        for part in msg.iter_parts():
            if part.get_content_type() == "text/plain":
                email_info["Body"] = part.get_content().strip()
                break
    else:
        # if not multipart get content directly
        email_info["Body"] = msg.get_content().strip()
    
    return email_info

# Extract raw email content
first_email = raw_messages[3757]
raw_email_content = first_email[b'BODY[]']

# Parse the email
parsed_email = parse_email(raw_email_content)

# Print parsed email information
for key, value in parsed_email.items():
    print(f"{key}: {value}")


Subject: ISRO posted: Aditya-L1: Celebration of First Orbit Completion 🌞🛰️ ...
From: "X (formerly Twitter)" <info@twitter.com>
To: Jamiu Oladipupo <jmldpp@gmail.com>
Date: Wed, 03 Jul 2024 03:14:48 +0000
Body: Your Highlights
ISRO - @isro
Aditya-L1: Celebration of First Orbit Completion 🌞🛰️
Today, Aditya-L1 completed its first halo orbit around the Sun-Earth L1 point. Inserted on January 6, 2024, it took 178 days, to complete a revolution.

Today's station-keeping manoeuvre ensured its seamless transition into… https://t.co/yB6vZQpIvE
Read more at Twitter <https://twitter.com/isro/status/1808122572263117306>
  ----------
Jared Isaacman - @rookisaacman
Appreciate the coverage @PopSci 

https://t.co/RRlrAILKUn
Read more at Twitter <https://twitter.com/rookisaacman/status/1808180384867537011>
  ----------
Solace - @Solace_PR
Send ur Acct details now⭐

N3m Giveaway, Crediting first 5000 likes, Rt and follow Me. No wizkid Fc should skip ❤️🦅 Turkey x Sszn ⭐ https://t.co/LRtWectEsI
Read more 